<a href="https://colab.research.google.com/github/ariahosseini/DeepML/blob/main/013_TensorFlow_Proj_Thirteen_GNN_Spektral_NodeLevel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install
!pip install spektral
!pip install ogb

In [ ]:
# utils
import numpy as np
from ogb.nodeproppred import Evaluator, NodePropPredDataset
# tensorflow
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.random import set_seed
# spektral
from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation, Cora
from spektral.datasets.ogb import OGB
from spektral.layers import GCNConv, ChebConv, GATConv, ARMAConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess, AdjToSpTensor, GCNFilter
from spektral.utils import tic, toc
# vis
import matplotlib.pyplot as plt


# Citation Data Using GCN

In [ ]:
# params
learning_rate = 1e-2
seed = 0
epochs = 200
patience = 10
data = "cora"

In [ ]:
set_seed(seed=seed)

In [ ]:
# load data
dataset = Citation(data, normalize_x=True, transforms=[LayerPreprocess(GCNConv)])

Pre-processing node features


In [ ]:
print(f"Data set name: {dataset.name}")
print(f"Data set type: {type(dataset)}")
print(f"No. of graphs: {dataset.n_graphs}")
print(f"No. of nodes: {dataset.n_nodes}")
print(f"No. of classes: {dataset.n_labels}")
print(f"No. of edge features: {dataset.n_edge_features}")
print(f"No. of node features: {dataset.n_node_features}")
print(f"Recap: {dataset.graphs}")

NameError: name 'dataset' is not defined

In [ ]:
print(f"Graph nodes' features:\n{dataset[0].x}")
print(f"Graph nodes' size:\n{np.shape(dataset[0].x)}")

Graph nodes' features:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Graph nodes' size:
(2708, 1433)


In [ ]:
print(f"Graph edges' features:\n{dataset[0].e}")

Graph edges' features:
None


In [ ]:
print(f"Graph adjacency matrix:\n{dataset[0].a}")

Graph adjacency matrix:
  (0, 0)	0.25
  (0, 633)	0.25
  (0, 1862)	0.2236068
  (0, 2582)	0.25
  (1, 1)	0.25
  (1, 2)	0.20412415
  (1, 652)	0.28867513
  (1, 654)	0.35355338
  (2, 1)	0.20412415
  (2, 2)	0.16666667
  (2, 332)	0.16666667
  (2, 1454)	0.28867513
  (2, 1666)	0.15430336
  (2, 1986)	0.05025189
  (3, 3)	0.49999997
  (3, 2544)	0.49999997
  (4, 4)	0.16666667
  (4, 1016)	0.16666667
  (4, 1256)	0.13608277
  (4, 1761)	0.14433756
  (4, 2175)	0.16666667
  (4, 2176)	0.13608277
  (5, 5)	0.25
  (5, 1629)	0.25
  (5, 1659)	0.28867513
  :	:
  (2699, 2699)	0.49999997
  (2700, 1151)	0.40824828
  (2700, 2700)	0.49999997
  (2701, 44)	0.28867513
  (2701, 2624)	0.3333333
  (2701, 2701)	0.3333333
  (2702, 186)	0.21821788
  (2702, 1536)	0.2581989
  (2702, 2702)	0.3333333
  (2703, 1298)	0.49999997
  (2703, 2703)	0.49999997
  (2704, 641)	0.49999997
  (2704, 2704)	0.49999997
  (2705, 287)	0.49999997
  (2705, 2705)	0.49999997
  (2706, 165)	0.19999999
  (2706, 169)	0.2581989
  (2706, 1473)	0.19999999
  (2

In [ ]:
print(f"Graph labels:\n{dataset[0].y}")
print(f"Graph labels size:\n{np.shape(dataset[0].y)}")

Graph labels:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Graph labels size:
(2708, 7)


In [ ]:
# convert the binary masks to sample weights so that one can compute the average loss over the nodes (following original implementation by Kipf & Welling)
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)
weights_tr, weights_va, weights_te = (mask_to_weights(mask) for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te))

In [ ]:
model = GCN(n_labels=dataset.n_labels, channels=16, activation='relu',
            output_activation='softmax', use_bias=False,
            dropout_rate=0.5, l2_reg=0.00025)
model.compile(optimizer=Adam(learning_rate),
              loss=CategoricalCrossentropy(reduction="sum"),
              weighted_metrics=["acc"])

In [ ]:
# train model
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
    verbose=0
    )

In [ ]:
model.summary()

Model: "gcn_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_2 (Dropout)         multiple                  0         
                                                                 
 gcn_conv_2 (GCNConv)        multiple                  22928     
                                                                 
 dropout_3 (Dropout)         multiple                  0         
                                                                 
 gcn_conv_3 (GCNConv)        multiple                  112       
                                                                 
Total params: 23040 (90.00 KB)
Trainable params: 23040 (90.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# evaluate model
print("Evaluating model.")
loader_va = SingleLoader(dataset, sample_weights=weights_va)
eval_results = model.evaluate(loader_va.load(), steps=loader_va.steps_per_epoch)
print("Done.\n" "Val loss: {}\n" "Val accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 31ms/step - loss: 1.0540 - acc: 0.7880
Done.
Val loss: 1.0539989471435547
Val accuracy: 0.7879999876022339


In [ ]:
# pred
print("Prediction.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.predict(loader_te.load(), steps=loader_te.steps_per_epoch)
output = [np.argmax(_) for _ in eval_results]
print(output)

Prediction.
1/1 [==============================] - 0s 25ms/step
[3, 4, 4, 0, 3, 2, 0, 3, 3, 2, 0, 0, 4, 3, 3, 3, 2, 3, 1, 3, 5, 3, 4, 6, 3, 3, 6, 3, 2, 4, 3, 6, 0, 4, 2, 0, 1, 5, 4, 4, 3, 6, 6, 4, 3, 3, 2, 5, 3, 4, 5, 3, 0, 2, 1, 4, 6, 3, 2, 2, 0, 0, 0, 4, 2, 0, 4, 5, 2, 6, 5, 2, 2, 2, 0, 4, 5, 6, 4, 0, 0, 0, 4, 2, 4, 1, 4, 6, 0, 4, 2, 4, 6, 6, 0, 0, 6, 5, 0, 6, 0, 2, 1, 1, 1, 2, 6, 5, 6, 1, 2, 2, 1, 5, 5, 5, 6, 5, 6, 5, 5, 1, 6, 6, 1, 5, 1, 6, 5, 5, 5, 1, 5, 1, 1, 1, 1, 1, 1, 1, 4, 4, 0, 3, 6, 6, 0, 6, 4, 0, 3, 4, 4, 1, 2, 2, 2, 3, 3, 3, 3, 6, 0, 5, 0, 3, 4, 0, 0, 3, 2, 3, 4, 2, 2, 6, 1, 4, 3, 3, 3, 6, 3, 3, 1, 3, 3, 4, 2, 2, 6, 1, 2, 5, 4, 0, 4, 3, 4, 4, 3, 3, 2, 4, 0, 3, 2, 3, 3, 4, 3, 0, 3, 6, 0, 3, 3, 4, 3, 3, 5, 2, 1, 2, 3, 6, 3, 2, 2, 3, 3, 3, 3, 5, 1, 3, 1, 3, 5, 0, 4, 5, 0, 4, 2, 4, 2, 4, 4, 5, 1, 3, 6, 3, 4, 6, 4, 0, 4, 5, 2, 3, 6, 2, 5, 5, 0, 2, 2, 3, 0, 4, 0, 3, 0, 4, 0, 0, 4, 0, 6, 3, 5, 4, 4, 6, 4, 1, 3, 2, 2, 4, 3, 4, 1, 3, 2, 3, 3, 4, 0, 2, 1, 1, 0, 0, 1, 6, 1, 4, 3, 3,

# Cora Data Using GCN Custom Traing Loop

In [ ]:
set_seed(seed=0)

In [ ]:
dataset = Cora(normalize_x=True, transforms=[LayerPreprocess(GCNConv), AdjToSpTensor()])

Pre-processing node features


/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
print(f"Data set name: {dataset.name}")
print(f"Data set type: {type(dataset)}")
print(f"No. of graphs: {dataset.n_graphs}")
print(f"No. of nodes: {dataset.n_nodes}")
print(f"No. of classes: {dataset.n_labels}")
print(f"No. of edge features: {dataset.n_edge_features}")
print(f"No. of node features: {dataset.n_node_features}")
print(f"Recap: {dataset.graphs}")

Data set name: cora
Data set type: <class 'spektral.datasets.citation.Cora'>
No. of graphs: 1
No. of nodes: 2708
No. of classes: 7
No. of edge features: None
No. of node features: 1433
Recap: [Graph(n_nodes=2708, n_node_features=1433, n_edge_features=None, n_labels=7)]


In [ ]:
graph = dataset[0]
nodes, adj_matrix, labels = graph.x, graph.a, graph.y

In [ ]:
mask_trian, mask_validation, mask_test = dataset.mask_tr, dataset.mask_va, dataset.mask_te

In [ ]:
model = GCN(n_labels=dataset.n_labels)
optimizer = Adam(learning_rate=1e-2)
loss_func = CategoricalCrossentropy()

In [ ]:
# training step
@tf.function
def train():
    with tf.GradientTape() as tape:
        predictions = model([nodes, adj_matrix], training=True)
        loss = loss_func(labels[mask_trian], predictions[mask_trian])
        loss += sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss
train()  # warm up to ignore tracing times when timing

<tf.Tensor: shape=(), dtype=float32, numpy=1.9543768>

In [ ]:
# time the execution of 200 epochs of training
tic()
for epoch in range(1, 201):
    loss = train()
toc("Spektral - GCN (200 epochs)")
print(f"Final loss = {loss}")

Spektral - GCN (200 epochs)
Elapsed: 12.84s
Final loss = 0.6075741648674011


# Cora Data Using ChebConv

In [ ]:
# load data
dataset = Citation("cora", transforms=[LayerPreprocess(ChebConv)])

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
def mask_to_weights(mask):
    return mask / np.count_nonzero(mask)
weights_train, weights_valid, weights_test = (mask_to_weights(mask) for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te))

In [ ]:
# parames
channels = 16  # number of channels in the first layer
cheb_deg = 2  # max degree of the Chebyshev polynomials
dropout = 0.5  # dropout rate for the features
l2_reg = 2.5e-4  # l2 regularization rate
learning_rate = 1e-2  # learning rate
epochs = 200  # number of training epochs
patience = 10  # patience for early stopping

In [ ]:
print(f"Data set name: {dataset.name}")
print(f"Data set type: {type(dataset)}")
print(f"No. of graphs: {dataset.n_graphs}")
print(f"No. of nodes: {dataset.n_nodes}")
print(f"No. of classes: {dataset.n_labels}")
print(f"No. of edge features: {dataset.n_edge_features}")
print(f"No. of node features: {dataset.n_node_features}")
print(f"Recap: {dataset.graphs}")

Data set name: cora
Data set type: <class 'spektral.datasets.citation.Citation'>
No. of graphs: 1
No. of nodes: 2708
No. of classes: 7
No. of edge features: None
No. of node features: 1433
Recap: [Graph(n_nodes=2708, n_node_features=1433, n_edge_features=None, n_labels=7)]


In [ ]:
# vars
adj_matrix_dtype = dataset[0].a.dtype  # only needed for TF 2.1
num_nodes = dataset.n_nodes  # number of nodes in the graph
num_feat = dataset.n_node_features  # original size of node features
num_labels = dataset.n_labels  # number of classes

In [ ]:
# define model
nodes_input = Input(shape=(num_feat,))
adj_matrix_input = Input((num_nodes,), sparse=True, dtype=adj_matrix_dtype)
x = Dropout(dropout)(nodes_input)
x = ChebConv(channels, K=cheb_deg, activation="relu", kernel_regularizer=l2(l2_reg), use_bias=False)([x, adj_matrix_input])
x = Dropout(dropout)(x)
outputs = ChebConv(num_labels, K=cheb_deg, activation="softmax", use_bias=False)([x, adj_matrix_input])

In [ ]:
# build model
model = Model(inputs=[nodes_input, adj_matrix_input], outputs=outputs)
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(reduction="sum"), weighted_metrics=["acc"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1433)]               0         []                            
                                                                                                  
 dropout_6 (Dropout)         (None, 1433)                 0         ['input_1[0][0]']             
                                                                                                  
 input_2 (InputLayer)        [(None, 2708)]               0         []                            
                                                                                                  
 cheb_conv (ChebConv)        (None, 16)                   45856     ['dropout_6[0][0]',           
                                                                     'input_2[0][0]']         

In [ ]:
# train model
loader_train = SingleLoader(dataset, sample_weights=weights_train)
loader_valid = SingleLoader(dataset, sample_weights=weights_valid)
model.fit(
    loader_train.load(),
    steps_per_epoch=loader_train.steps_per_epoch,
    validation_data=loader_valid.load(),
    validation_steps=loader_valid.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)]
    )

Epoch 1/200
1/1 [==============================] - 1s 850ms/step - loss: 1.9632 - acc: 0.1286 - val_loss: 1.9147 - val_acc: 0.2940
Epoch 2/200
1/1 [==============================] - 0s 118ms/step - loss: 1.8490 - acc: 0.3429 - val_loss: 1.8765 - val_acc: 0.3880
Epoch 3/200
1/1 [==============================] - 0s 163ms/step - loss: 1.7146 - acc: 0.5357 - val_loss: 1.8185 - val_acc: 0.4320
Epoch 4/200
1/1 [==============================] - 0s 159ms/step - loss: 1.5424 - acc: 0.6000 - val_loss: 1.7451 - val_acc: 0.4780
Epoch 5/200
1/1 [==============================] - 0s 161ms/step - loss: 1.3844 - acc: 0.6929 - val_loss: 1.6660 - val_acc: 0.5200
Epoch 6/200
1/1 [==============================] - 0s 130ms/step - loss: 1.1900 - acc: 0.7429 - val_loss: 1.5808 - val_acc: 0.5700
Epoch 7/200
1/1 [==============================] - 0s 152ms/step - loss: 1.0213 - acc: 0.8071 - val_loss: 1.4939 - val_acc: 0.6000
Epoch 8/200
1/1 [==============================] - 0s 159ms/step - loss: 0.9343 - a

In [ ]:
# evaluate model
print("Evaluating model.")
loader_test = SingleLoader(dataset, sample_weights=weights_test)
eval_results = model.evaluate(loader_test.load(), steps=loader_test.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 39ms/step - loss: 0.8185 - acc: 0.7770
Done.
Test loss: 0.8184532523155212
Test accuracy: 0.7769999504089355


# Cora Data Using GAT

In [ ]:
set_seed(0)

In [ ]:
# load data
dataset = Citation("cora", normalize_x=True, transforms=[LayerPreprocess(GATConv)])

Pre-processing node features


/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)
weights_train, weights_validation, weights_test = (mask_to_weights(mask) for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te))

In [ ]:
# params
channels = 8  # number of channels in each head of the first GAT layer
num_attn_heads = 8  # number of attention heads in first GAT layer
dropout = 0.6  # dropout rate for the features and adjacency matrix
l2_reg = 2.5e-4  # l2 regularization rate
learning_rate = 5e-3  # learning rate
epochs = 20  # number of training epochs
patience = 100  # patience for early stopping

In [ ]:
# vars
num_nodes = dataset.n_nodes  # number of nodes in the graph
num_feat = dataset.n_node_features  # original size of node features
num_labels = dataset.n_labels  # number of classes

In [ ]:
# define model
input = Input(shape=(num_feat,))
adj_mat = Input((num_nodes,), sparse=True)
x = Dropout(dropout)(input)
x = GATConv(
    channels,
    attn_heads=num_attn_heads,
    concat_heads=True,
    dropout_rate=dropout,
    activation="elu",
    kernel_regularizer=l2(l2_reg),
    attn_kernel_regularizer=l2(l2_reg),
    bias_regularizer=l2(l2_reg)
    )([x, adj_mat])
x = Dropout(dropout)(x)
output = GATConv(
    num_labels,
    attn_heads=1,
    concat_heads=False,
    dropout_rate=dropout,
    activation="softmax",
    kernel_regularizer=l2(l2_reg),
    attn_kernel_regularizer=l2(l2_reg),
    bias_regularizer=l2(l2_reg)
    )([x, adj_mat])

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [ ]:
# build model
model = Model(inputs=[input, adj_mat], outputs=output)
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(reduction="sum"),
    weighted_metrics=["acc"])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 1433)]               0         []                            
                                                                                                  
 dropout_10 (Dropout)        (None, 1433)                 0         ['input_3[0][0]']             
                                                                                                  
 input_4 (InputLayer)        [(None, 2708)]               0         []                            
                                                                                                  
 gat_conv (GATConv)          (None, 64)                   91904     ['dropout_10[0][0]',          
                                                                     'input_4[0][0]']       

In [ ]:
# train model
loader_train = SingleLoader(dataset, sample_weights=weights_train)
loader_valid = SingleLoader(dataset, sample_weights=weights_validation)
model.fit(
    loader_train.load(),
    steps_per_epoch=loader_train.steps_per_epoch,
    validation_data=loader_valid.load(),
    validation_steps=loader_valid.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)]
    )

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 1.9496 - acc: 0.2143 - val_loss: 1.9486 - val_acc: 0.0740
Epoch 2/20
1/1 [==============================] - 0s 85ms/step - loss: 1.9471 - acc: 0.2214 - val_loss: 1.9468 - val_acc: 0.0740
Epoch 3/20
1/1 [==============================] - 0s 91ms/step - loss: 1.9444 - acc: 0.2429 - val_loss: 1.9454 - val_acc: 0.1440
Epoch 4/20
1/1 [==============================] - 0s 87ms/step - loss: 1.9432 - acc: 0.3214 - val_loss: 1.9446 - val_acc: 0.3560
Epoch 5/20
1/1 [==============================] - 0s 82ms/step - loss: 1.9381 - acc: 0.4357 - val_loss: 1.9428 - val_acc: 0.4700
Epoch 6/20
1/1 [==============================] - 0s 84ms/step - loss: 1.9347 - acc: 0.4714 - val_loss: 1.9408 - val_acc: 0.4600
Epoch 7/20
1/1 [==============================] - 0s 87ms/step - loss: 1.9345 - acc: 0.4500 - val_loss: 1.9384 - val_acc: 0.5980
Epoch 8/20
1/1 [==============================] - 0s 87ms/step - loss: 1.9334 - acc: 0.4500 - val_l

In [ ]:
# evaluate model
print("Evaluating model.")
loader_test = SingleLoader(dataset, sample_weights=weights_test)
eval_results = model.evaluate(loader_test.load(), steps=loader_test.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 29ms/step - loss: 1.9014 - acc: 0.7450
Done.
Test loss: 1.9014073610305786
Test accuracy: 0.7450000047683716


# Cora Data Using GAT Custom Traing Loop

In [ ]:
set_seed(0)

In [ ]:
# load data
dataset = Cora(normalize_x=True, transforms=[LayerPreprocess(GATConv), AdjToSpTensor()])

Pre-processing node features


In [ ]:
# graph
graph = dataset[0]
nodes, adj_matrix, labels = graph.x, graph.a, graph.y

In [ ]:
# weights
mask_train, mask_validation, mask_test = dataset.mask_tr, dataset.mask_va, dataset.mask_te

In [ ]:
# params
channels = 8  # number of channels in each head of the first GAT layer
num_attn_heads = 8  # number of attention heads in first GAT layer
dropout = 0.6  # dropout rate for the features and adjacency matrix
l2_reg = 2.5e-4  # l2 regularization rate
learning_rate = 5e-3  # learning rate
epochs = 20  # number of training epochs
patience = 100  # patience for early stopping

In [ ]:
# vars
num_nodes = dataset.n_nodes  # number of nodes in the graph
num_feat = dataset.n_node_features  # original size of node features
num_labels = dataset.n_labels  # number of classes

In [ ]:
# define model
input = Input(shape=(num_feat,))
adj_mat = Input((num_nodes,), sparse=True)
x = Dropout(dropout)(input)
x = GATConv(
    channels,
    attn_heads=num_attn_heads,
    concat_heads=True,
    dropout_rate=dropout,
    activation="elu",
    kernel_regularizer=l2(l2_reg),
    attn_kernel_regularizer=l2(l2_reg),
    bias_regularizer=l2(l2_reg)
    )([x, adj_mat])
x = Dropout(dropout)(x)
output = GATConv(
    num_labels,
    attn_heads=1,
    concat_heads=False,
    dropout_rate=dropout,
    activation="softmax",
    kernel_regularizer=l2(l2_reg),
    attn_kernel_regularizer=l2(l2_reg),
    bias_regularizer=l2(l2_reg)
    )([x, adj_mat])

In [ ]:
# build model
model = Model(inputs=[input, adj_mat], outputs=output)
optimizer = Adam(learning_rate=learning_rate)
loss_func = CategoricalCrossentropy()

In [ ]:
# training step
@tf.function
def train():
    with tf.GradientTape() as tape:
        predictions = model([nodes, adj_matrix], training=True)
        loss = loss_func(labels[mask_train], predictions[mask_train])
        loss += sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

@tf.function
def evaluate():
    predictions = model([nodes, adj_matrix], training=False)
    losses = []
    accuracies = []
    for mask in [mask_train, mask_validation, mask_test]:
        loss = loss_func(labels[mask], predictions[mask])
        loss += sum(model.losses)
        losses.append(loss)
        acc = tf.reduce_mean(categorical_accuracy(labels[mask], predictions[mask]))
        accuracies.append(acc)
    return losses, accuracies

best_val_loss = 1e6
best_test_acc = 0
current_patience = patience = 100
epochs = 100
tic()
for epoch in range(1, epochs + 1):
    train()
    losses, accu = evaluate()
    print(
        "Loss train: {:.4f}, Acc train: {:.4f}, "
        "Loss validation: {:.4f}, Acc validation: {:.4f}, "
        "Loss test: {:.4f}, Acc test: {:.4f}".format(losses[0], accu[0], losses[1], accu[1], losses[2], accu[2])
        )
    if losses[1] < best_val_loss:
        best_val_loss = losses[1]
        best_test_acc = accu[2]
        current_patience = patience
        print("Improved")
    else:
        current_patience -= 1
        if current_patience == 0:
            print("Test accuracy: {}".format(best_test_acc))
            break
toc("GAT ({} epochs)".format(epoch))

Loss train: 1.9467, Acc train: 0.3643, Loss validation: 1.9495, Acc validation: 0.2300, Loss test: 1.9497, Acc test: 0.2070
Improved
Loss train: 1.9441, Acc train: 0.6357, Loss validation: 1.9487, Acc validation: 0.4080, Loss test: 1.9486, Acc test: 0.3880
Improved
Loss train: 1.9416, Acc train: 0.7000, Loss validation: 1.9466, Acc validation: 0.4740, Loss test: 1.9465, Acc test: 0.4430
Improved
Loss train: 1.9392, Acc train: 0.7143, Loss validation: 1.9444, Acc validation: 0.3960, Loss test: 1.9442, Acc test: 0.4140
Improved
Loss train: 1.9366, Acc train: 0.5857, Loss validation: 1.9421, Acc validation: 0.3120, Loss test: 1.9419, Acc test: 0.3310
Improved
Loss train: 1.9339, Acc train: 0.5286, Loss validation: 1.9400, Acc validation: 0.2980, Loss test: 1.9399, Acc test: 0.3280
Improved
Loss train: 1.9310, Acc train: 0.6714, Loss validation: 1.9382, Acc validation: 0.4860, Loss test: 1.9379, Acc test: 0.4720
Improved
Loss train: 1.9278, Acc train: 0.8286, Loss validation: 1.9365, Acc v

# Cora Data Using ARMA

In [ ]:
# load data
dataset = Citation("cora", transforms=[LayerPreprocess(ARMAConv)])

In [ ]:
# weights
mask_train, mask_validation, mask_test = dataset.mask_tr, dataset.mask_va, dataset.mask_te

In [ ]:
# params
channels = 16  # number of channels in the first layer
iterations = 1  # number of iterations to approximate each ARMA(1)
order = 2  # order of the ARMA filter (number of parallel stacks)
share_weights = True  # share weights in each ARMA stack
dropout_skip = 0.75  # dropout rate for the internal skip connection of ARMA
dropout = 0.5  # dropout rate for the features
l2_reg = 5e-5  # l2 regularization rate
learning_rate = 1e-2  # learning rate
epochs = 20  # number of training epochs
patience = 100  # patience for early stopping
adj_matrix_dtype = dataset[0].a.dtype  # only needed for TF 2.1

In [ ]:
# vars
num_nodes = dataset.n_nodes  # number of nodes in the graph
num_feat = dataset.n_node_features  # original size of node features
num_labels = dataset.n_labels  # number of classes

In [ ]:
# define model
input = Input(shape=(num_feat,))
adj_mat = Input((num_nodes,), sparse=True, dtype=adj_matrix_dtype)
x = ARMAConv(
    channels,
    iterations=iterations,
    order=order,
    share_weights=share_weights,
    dropout_rate=dropout_skip,
    activation="elu",
    gcn_activation="elu",
    kernel_regularizer=l2(l2_reg)
    )([input, adj_mat])
x = Dropout(dropout)(x)
output = ARMAConv(
    num_labels,
    iterations=1,
    order=1,
    share_weights=share_weights,
    dropout_rate=dropout_skip,
    activation="softmax",
    gcn_activation=None,
    kernel_regularizer=l2(l2_reg)
    )([x, adj_mat])

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [ ]:
# build model
model = Model(inputs=[input, adj_mat], outputs=output)
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss="categorical_crossentropy",
              weighted_metrics=["acc"])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 1433)]               0         []                            
                                                                                                  
 input_8 (InputLayer)        [(None, 2708)]               0         []                            
                                                                                                  
 arma_conv (ARMAConv)        (None, 16)                   91744     ['input_7[0][0]',             
                                                                     'input_8[0][0]']             
                                                                                                  
 dropout_14 (Dropout)        (None, 16)                   0         ['arma_conv[0][0]']     

In [ ]:
# train model
loader_train = SingleLoader(dataset, sample_weights=weights_train)
loader_valid = SingleLoader(dataset, sample_weights=weights_valid)
model.fit(
    loader_train.load(),
    steps_per_epoch=loader_train.steps_per_epoch,
    validation_data=loader_valid.load(),
    validation_steps=loader_valid.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)]
    )

Epoch 1/20
1/1 [==============================] - 1s 1s/step - loss: 0.0083 - acc: 0.1429 - val_loss: 0.0067 - val_acc: 0.1640
Epoch 2/20
1/1 [==============================] - 0s 50ms/step - loss: 0.0067 - acc: 0.1929 - val_loss: 0.0052 - val_acc: 0.2220
Epoch 3/20
1/1 [==============================] - 0s 47ms/step - loss: 0.0053 - acc: 0.2357 - val_loss: 0.0040 - val_acc: 0.2740
Epoch 4/20
1/1 [==============================] - 0s 46ms/step - loss: 0.0041 - acc: 0.2786 - val_loss: 0.0031 - val_acc: 0.3840
Epoch 5/20
1/1 [==============================] - 0s 48ms/step - loss: 0.0031 - acc: 0.4000 - val_loss: 0.0024 - val_acc: 0.5000
Epoch 6/20
1/1 [==============================] - 0s 54ms/step - loss: 0.0024 - acc: 0.4643 - val_loss: 0.0020 - val_acc: 0.6120
Epoch 7/20
1/1 [==============================] - 0s 49ms/step - loss: 0.0020 - acc: 0.4857 - val_loss: 0.0018 - val_acc: 0.6960
Epoch 8/20
1/1 [==============================] - 0s 58ms/step - loss: 0.0017 - acc: 0.6143 - val_l

In [ ]:
# evaluate model
print("Evaluating model.")
loader_test = SingleLoader(dataset, sample_weights=mask_test)
eval_results = model.evaluate(loader_test.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 148ms/step - loss: 0.6251 - acc: 0.7600
Done.
Test loss: 0.6251000165939331
Test accuracy: 0.7599999904632568


# Citation networks SimpleGCN Custom Transform

In [ ]:
# define SGCN
class SGCN:
    def __init__(self, K):
        self.K = K # propagation steps for SGCN
    def __call__(self, graph):
        output = graph.a
        for _ in range(self.K-1):
            output = output.dot(output)
        output.sort_indices()
        graph.a = output
        return graph

In [ ]:
# load data
K = 2  # propagation steps for SGCN
dataset = Citation("cora", transforms=[LayerPreprocess(GCNConv), SGCN(K)])

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
# weights
mask_train, mask_validation, mask_test = dataset.mask_tr, dataset.mask_va, dataset.mask_te

In [ ]:
# params
l2_reg = 5e-6  # L2 regularization rate
learning_rate = 0.2  # learning rate
epochs = 20  # number of training epochs
patience = 200  # patience for early stopping
adj_matrix_dtype = dataset[0].a.dtype  # only needed for TF 2.1

In [ ]:
# vars
num_nodes = dataset.n_nodes  # number of nodes in the graph
num_feat = dataset.n_node_features  # original size of node features
num_labels = dataset.n_labels  # number of classes

In [ ]:
# define model
input = Input(shape=(num_feat,))
adj_mat = Input((num_nodes,), sparse=True, dtype=adj_matrix_dtype)
output = GCNConv(num_labels, activation="softmax",
                 kernel_regularizer=l2(l2_reg), use_bias=False)([input, adj_mat])

In [ ]:
# build model
model = Model(inputs=[input, adj_mat], outputs=output)
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", weighted_metrics=["acc"])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1433)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 2708)]               0         []                            
                                                                                                  
 gcn_conv (GCNConv)          (None, 7)                    10031     ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
Total params: 10031 (39.18 KB)
Trainable params: 10031 (39.18 KB)
Non-trainable params: 0 (0

In [ ]:
# train model
loader_train = SingleLoader(dataset, sample_weights=weights_train)
loader_valid = SingleLoader(dataset, sample_weights=weights_valid)
model.fit(
    loader_train.load(),
    steps_per_epoch=loader_train.steps_per_epoch,
    validation_data=loader_valid.load(),
    validation_steps=loader_valid.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)]
    )

Epoch 1/20
1/1 [==============================] - 0s 393ms/step - loss: 7.9237e-04 - acc: 0.1000 - val_loss: 6.5436e-04 - val_acc: 0.6820
Epoch 2/20
1/1 [==============================] - 0s 27ms/step - loss: 5.3699e-04 - acc: 0.9000 - val_loss: 6.4911e-04 - val_acc: 0.7580
Epoch 3/20
1/1 [==============================] - 0s 27ms/step - loss: 4.8026e-04 - acc: 0.9571 - val_loss: 7.0950e-04 - val_acc: 0.7700
Epoch 4/20
1/1 [==============================] - 0s 29ms/step - loss: 5.2773e-04 - acc: 0.9786 - val_loss: 7.3929e-04 - val_acc: 0.7700
Epoch 5/20
1/1 [==============================] - 0s 23ms/step - loss: 5.5492e-04 - acc: 0.9857 - val_loss: 7.1382e-04 - val_acc: 0.7680
Epoch 6/20
1/1 [==============================] - 0s 28ms/step - loss: 5.3130e-04 - acc: 0.9857 - val_loss: 6.6454e-04 - val_acc: 0.7600
Epoch 7/20
1/1 [==============================] - 0s 25ms/step - loss: 4.9043e-04 - acc: 0.9857 - val_loss: 6.2570e-04 - val_acc: 0.7560
Epoch 8/20
1/1 [========================

In [ ]:
# Evaluate model
print("Evaluating model.")
loader_test = SingleLoader(dataset, sample_weights=mask_test)
eval_results = model.evaluate(loader_test.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 104ms/step - loss: 0.4532 - acc: 0.8020
Done.
Test loss: 0.4532226026058197
Test accuracy: 0.8019999861717224


# Open Graph Benchmark Dataset

In [ ]:
clear_session()

In [ ]:
# load data
dataset_name = "ogbn-arxiv"
ogb_dataset = NodePropPredDataset(dataset_name)
dataset = OGB(ogb_dataset, transforms=[GCNFilter(), AdjToSpTensor()])

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:05<00:00, 15.08it/s]


Extracting dataset/arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 1665.07it/s]

Saving...


In [ ]:
# graph
graph = dataset[0]
nodes, adj_matrix, labels = graph.x, graph.a, graph.y

In [ ]:
# params
channels = 256  # number of channels for GCN layers
dropout = 0.5  # dropout rate for the features
learning_rate = 1e-2  # learning rate
epochs = 20  # number of training epochs

In [ ]:
# vars
num_nodes = dataset.n_nodes  # number of nodes in the graph
num_feat = dataset.n_node_features  # original size of node features
num_labels = ogb_dataset.num_classes  # number of classes

In [ ]:
# data splits
idx = ogb_dataset.get_idx_split()
idx_train, idx_val, idx_test = idx["train"], idx["valid"], idx["test"]
mask_train = np.zeros(num_nodes, dtype=bool)
mask_valid = np.zeros(num_nodes, dtype=bool)
mask_test = np.zeros(num_nodes, dtype=bool)
mask_train[idx_train] = True
mask_valid[idx_val] = True
mask_test[idx_test] = True
masks = [mask_train, mask_valid, mask_test]

In [ ]:
# define model
node_input = Input(shape=(num_feat,))
adj_mat_input = Input((num_nodes,), sparse=True)
x = GCNConv(channels, activation="relu")([node_input, adj_mat_input])
x = BatchNormalization()(x)
x = Dropout(dropout)(x)
x = GCNConv(channels, activation="relu")([x, adj_mat_input])
x = BatchNormalization()(x)
x = Dropout(dropout)(x)
output = GCNConv(num_labels, activation="softmax")([x, adj_mat_input])

In [ ]:
# build model
model = Model(inputs=[node_input, adj_mat_input], outputs=output)
optimizer = Adam(learning_rate=learning_rate)
loss_func = SparseCategoricalCrossentropy()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 169343)]             0         []                            
                                                                                                  
 gcn_conv (GCNConv)          (None, 256)                  33024     ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256)                  1024      ['gcn_conv[0][0]']        

In [ ]:
# training function
@tf.function
def train(inputs, target, mask):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_func(target[mask], predictions[mask]) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [ ]:
# evaluation with OGB
evaluator = Evaluator(dataset_name)

In [ ]:
# train and evaluate model
def evaluate(nodes, adj_mat, labels, model, masks, evaluator):
    pred = model([nodes, adj_mat], training=False)
    pred = pred.numpy().argmax(-1)[:, None]
    train_mask, val_mask, test_mask = masks
    train_auc = evaluator.eval({"y_true": labels[train_mask], "y_pred": pred[train_mask]})["acc"]
    val_auc = evaluator.eval({"y_true": labels[val_mask], "y_pred": pred[val_mask]})["acc"]
    test_auc = evaluator.eval({"y_true": labels[test_mask], "y_pred": pred[test_mask]})["acc"]
    return train_auc, val_auc, test_auc

for i in range(1, 1 + epochs):
    train_loss = train([nodes, adj_matrix], labels, mask_train)
    train_acc, val_acc, test_acc = evaluate(nodes, adj_matrix, labels, model, masks, evaluator)
    print(
        "Ep. {} - Loss: {:.3f} - Acc: {:.3f} - Val acc: {:.3f} - Test acc: "
        "{:.3f}".format(i, train_loss, train_acc, val_acc, test_acc)
        )
print("Evaluating model!")
train_acc, val_acc, test_acc = evaluate(nodes, adj_matrix, labels, model, masks, evaluator)
print("Done! - Test acc: {:.3f}".format(test_acc))

Ep. 1 - Loss: 5.482 - Acc: 0.186 - Val acc: 0.336 - Test acc: 0.343
Ep. 2 - Loss: 3.007 - Acc: 0.180 - Val acc: 0.193 - Test acc: 0.206
Ep. 3 - Loss: 2.619 - Acc: 0.224 - Val acc: 0.274 - Test acc: 0.332
Ep. 4 - Loss: 2.248 - Acc: 0.153 - Val acc: 0.165 - Test acc: 0.158
Ep. 5 - Loss: 2.143 - Acc: 0.124 - Val acc: 0.139 - Test acc: 0.134
Ep. 6 - Loss: 2.002 - Acc: 0.115 - Val acc: 0.129 - Test acc: 0.124
Ep. 7 - Loss: 1.932 - Acc: 0.074 - Val acc: 0.064 - Test acc: 0.059
Ep. 8 - Loss: 1.884 - Acc: 0.042 - Val acc: 0.032 - Test acc: 0.024
Ep. 9 - Loss: 1.827 - Acc: 0.029 - Val acc: 0.021 - Test acc: 0.013
Ep. 10 - Loss: 1.799 - Acc: 0.019 - Val acc: 0.012 - Test acc: 0.007
Ep. 11 - Loss: 1.768 - Acc: 0.012 - Val acc: 0.007 - Test acc: 0.004
Ep. 12 - Loss: 1.740 - Acc: 0.006 - Val acc: 0.004 - Test acc: 0.002
Ep. 13 - Loss: 1.715 - Acc: 0.004 - Val acc: 0.003 - Test acc: 0.002
Ep. 14 - Loss: 1.702 - Acc: 0.003 - Val acc: 0.002 - Test acc: 0.001
Ep. 15 - Loss: 1.678 - Acc: 0.003 - Val acc